## Sub Task 1: Parse all tweets of handle *midasIIITD*

### import required libraries
- json: for saving and loading of data 
- requests: for sending GET request to Twitter API 
- requests_oauthlib: for authentication on Twitter API 

In [1]:
import sys
import json
import requests
from requests_oauthlib import OAuth1

Authentication for access of Twitter API
- Keys had been removed

In [9]:
def get_authenticated():

    CONSUMER_KEY = ""
    CONSUMER_SECRET = ""
    ACCESS_KEY = ""
    ACCESS_SECRET = ""

    auth = OAuth1(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_KEY, ACCESS_SECRET)

    return auth


- Since twitter by default allow only 20 tweets on one request if count parameters is not given any value. Therefore in one request we have to get the total no. of tweets/retweets by an user.
- **get_count_of_tweet** accepts 3 parameter *user_handle*, base *URL* and *auth* object and returns total no. of tweets by an user

In [3]:
def get_count_of_tweet(user_handle, URL, auth):

    URL = URL + "?screen_name={}".format(user_handle)

    try:
        response = requests.get(URL, auth=auth)
    except requests.exceptions.RequestException:
        print('No internet connection')
        sys.exit(1)

    data = response.json()
    count = data[0]["user"]["statuses_count"]

    return count


**get_json_data** send the GET requests for fetching the tweets. This function accepts _userhandle_, _nooftweets_, and base _URL_ and returns the parsed **JSON** data

In [4]:
def get_json_data(user_handle, URL, no_of_tweets, auth):

    URL = URL + \
        "?screen_name={}&tweet_mode=extended&count={}".format(
            user_handle, no_of_tweets)

    try:
        response = requests.get(URL, auth=auth)
    except requests.exceptions.RequestException:
        print('No internet connection')
        sys.exit(1)

    data = response.json()

    return data


**save_data_into_file** accepts the JSON data and save it into file named **tweet.json**

In [5]:
def save_data_into_file(data):

    with open('tweet.json', 'w') as f:
        json.dump(data, f, indent=4)


**main** calls all the required function and finish the subtask

In [6]:
def main():

    user_handle = "midasIIITD"
    URL = 'https://api.twitter.com/1.1/statuses/user_timeline.json'

    auth = get_authenticated()
    no_of_tweets = get_count_of_tweet(user_handle, URL, auth)
    data = get_json_data(user_handle, URL, no_of_tweets, auth)
    save_data_into_file(data)


if __name__ == '__main__':
    main()


## Sub Task 2: Load tweets from JSON file and print it in a tabular form

**BeautifulTable** allow to print data in tabular manner this block import neccessary lib and print the tweets in tabular form

In [7]:
def get_all_media_count(tweet):
    
    count = 0
    
    if 'extended_entities' in tweet:
        try:
            count += len(tweet['extended_entities']['media'])
        except KeyError:
            pass
    
    elif 'media' in tweet['entities']:
        count += len(tweet['entities']['media'])
    
    # Need to check if there is media available in retweet
    if 'retweeted_status' in tweet:
        count += get_all_media_count(tweet['retweeted_status'])
    
    return count

In [8]:
from beautifultable import BeautifulTable
table = BeautifulTable()
table.column_headers = ['Text', 'Date and Time', 'No. of favorites', 'No. of Retweets', 'No. of images']

with open('tweet.json', 'r') as f:
    json_data = f.read()

data = json.loads(json_data)

for tweet in data:
        
    count = get_all_media_count(tweet)
    if count == 0:
        count = None

    tweet_data = [tweet['full_text'], tweet['created_at'], tweet['favorite_count'], tweet['retweet_count'],
                  count]
    
    table.append_row(tweet_data)
    
print(table)



+---------------------------------------------------+-------+-----+-------+----+
|                       Text                        | Date  | No. | No. o | No |
|                                                   | and T |  of | f Ret | .  |
|                                                   |  ime  |  fa | weets | of |
|                                                   |       | vor |       |  i |
|                                                   |       | ite |       | ma |
|                                                   |       |  s  |       | ge |
|                                                   |       |     |       | s  |
+---------------------------------------------------+-------+-----+-------+----+
| Congratulations @midasIIITD students Simra Shahid | Mon M |  5  |   1   | 2  |
|  @Simcyy and Nilay Shrivastava @NilayShri on gett | ar 25 |     |       |    |
| ing selected for a research internship at Adobe i |  13:0 |     |       |    |
|                  n this su